In [1]:
# !! Important !!
# Set small_run = False, to run this feature engineering notebook for entire dataframe
# Setting small_run = True, runs the notebook only for top 100 rows of the dataframe
# I have added this flag so that this notebook can be executed in kaggle kernal

run_for_small_data = True

In [3]:
# import the required libraries 

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing import sequence, text
from keras.layers import Input, Embedding

from nltk import word_tokenize
from nltk.corpus import stopwords
from textblob import TextBlob

import datetime as dt
import pandas as pd
import numpy as np
import warnings
import string

# stop_words = []
stop_words = list(set(stopwords.words('english')))
warnings.filterwarnings('ignore')
punctuation = string.punctuation

In [6]:
# read data files 

id_column = "id"
missing_token = " UNK "

fdir  = '/Users/naveen/Dropbox/OnlineWork/Datasets/DonorsChooseApplicationScreening/input/'

train = pd.read_csv(fdir+"train.csv", parse_dates=["project_submitted_datetime"])
test = pd.read_csv(fdir+"test.csv", parse_dates=["project_submitted_datetime"])
rc = pd.read_csv(fdir+"resources.csv").fillna(missing_token)

df = pd.concat([train, test], axis=0)
print(train.shape)
print(test.shape)
print(rc.shape)


(182080, 16)
(78035, 15)
(1541272, 4)


In [8]:
rc.head()

,id,description,quantity,price
0,p233245,LC652 - Lakeshore Double-Space Mobile Drying Rack,1,149.00
1,p069063,Bouncy Bands for Desks (Blue support pipes),3,14.95
2,p069063,Cory Stories: A Kid's Book About Living With Adhd,1,8.45
3,p069063,"Dixon Ticonderoga Wood-Cased #2 HB Pencils, Bo...",2,13.59
4,p069063,EDUCATIONAL INSIGHTS FLUORESCENT LIGHT FILTERS...,3,24.95


In [15]:
rc['total_price'] = rc['quantity']*rc['price']
agg_rc = rc.groupby('id').agg({'description':'count', 'quantity':'sum', 'price':'sum', 'total_price':'sum'}).rename(columns={'description':'items'})

for func in ['min', 'max', 'mean']:
    agg_rc_temp = rc.groupby('id').agg({'quantity':func, 'price':func, 'total_price':func}).rename(columns={'quantity':func+'_quantity', 'price':func+'_price', 'total_price':func+'_total_price'}).fillna(0)
    agg_rc = agg_rc.join(agg_rc_temp)

print(agg_rc.head())

         items  quantity    price  total_price  min_quantity  min_price  \
id                                                                        
p000001      4         7   459.56       833.63             1      23.99   
p000002     14        21   515.89       630.28             1       8.46   
p000003      4         4   298.97       298.97             1      39.99   
p000004     95        98  1113.69      1126.22             1       1.60   
p000005      4         8   485.99       702.31             1      54.08   

         min_total_price  max_quantity  max_price  max_total_price  \
id                                                                   
p000001            47.98             2     261.08           522.16   
p000002             9.95             4     134.90           134.90   
p000003            39.99             1     169.00           169.00   
p000004             3.20             2     401.54           401.54   
p000005            54.08             3     323.75     

In [ ]:
agg_rc = agg_rc.join(rc.groupby('id').agg({'description':lambda x:' '.join(x.values.astype(str))}).rename(columns={'description':'resource_description'}))

df = df.join(agg_rc, on='id')

if run_for_small_data:
    df = df.head(100)

In [16]:
rc.groupby('id').agg({'description':lambda x:' '.join(x.values.astype(str))}).rename(columns={'description':'resource_description'})

,resource_description
id,
p000001,"Cap Barbell 300 Pound Olympic Set, Grey Cap Ba..."
p000002,10 Sony Headphones (BUY 9 GET 1 FREE) Belkin 6...
p000003,EE820X - Phonemic Awareness Instant Learning C...
p000004,A Bad Case of the Giggles Poems That Will Make...
p000005,"Fitbit Zip Wireless Activity Tracker, Lime Fit..."
p000006,Danny and the Dinosaur: 50th Anniversary Editi...
p000007,HP Chromebook 11 G4 11.6 Inch Laptop (Intel N2...
p000008,A Whale of a Tale Alvin and the Chipmunks Alvi...
p000009,Ball Don't Lie Baseball Card Adventure (4 Bk S...
